# Modelo ML XGBOOST CLASSIFIER

In [6]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import joblib
import json

# ========================
# A. Cargar datos
# ========================
X_train = pd.read_csv("../models/x_train_sel.csv")
y_train = pd.read_excel("../data/processed/X&Ys/y_train.xlsx").squeeze()

# ========================
# B. Cargar mapping sin modificarlo
# ========================
with open("../data/processed/Json/ciudad_transformation_rules.json") as f:
    ciudad_mapping = json.load(f)  # formato: {nombre: id}

# Invertir el mapping para usarlo
id_to_ciudad = {str(v): k for k, v in ciudad_mapping.items()}
y_train_nombres = y_train.astype(str).map(id_to_ciudad)

# ========================
# C. Codificar nombres
# ========================
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train_nombres)

# ========================
# D. Asegurarse de que las columnas numéricas estén incluidas
# ========================
columnas_extra = [
    "estimated_price_eur_x",
    "estimated_price_eur_y",
    "distance_to_city_center_km",
    "class_n"
]

columnas_finales = columnas_extra + [
    col for col in X_train.columns
    if col not in columnas_extra  # Evita duplicados
]

X_train_final = X_train[columnas_finales]

# ========================
# E. Entrenar modelo
# ========================
model = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")
model.fit(X_train_final, y_train_encoded)

# ========================
# F. Evaluar
# ========================
y_pred = model.predict(X_train_final)
print("✔️ Accuracy:", accuracy_score(y_train_encoded, y_pred))

# ========================
# G. Guardar modelo y encoder
# ========================
joblib.dump(model, "../models/model_completo.pkl")
joblib.dump(le, "../models/label_encoder.pkl")
pd.Series(X_train_final.columns).to_csv("../data/processed/x_train_columns.csv", index=False, header=False)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [10:45:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✔️ Accuracy: 1.0


# 🧪 PREDICCIÓN PERSONALIZADA

In [5]:
import pandas as pd
import numpy as np
import joblib
import json

# ========================
# 1. Cargar modelo y utilidades
# ========================
model = joblib.load("../models/model_completo.pkl")
le = joblib.load("../models/label_encoder.pkl")
columnas_modelo = pd.read_csv("../data/processed/x_train_columns.csv", header=None).squeeze().tolist()

with open("../data/processed/Json/ciudad_transformation_rules.json") as f:
    ciudad_mapping = json.load(f)
id_to_ciudad = {str(v): k for k, v in ciudad_mapping.items()}

full_df = pd.read_csv("../data/processed/total_data_240k.csv")

# ========================
# 2. Funciones auxiliares
# ========================
def construir_input_usuario(valores_dict, columnas_modelo):
    df = pd.DataFrame(columns=columnas_modelo)
    df.loc[0] = 0
    for clave, valor in valores_dict.items():
        col = f"{clave}_{valor}"
        if col in df.columns:
            df.at[0, col] = 1
    for col in ["estimated_price_eur_x", "estimated_price_eur_y", "distance_to_city_center_km"]:
        if col in valores_dict:
            df.at[0, col] = valores_dict[col]
    return df

def get_clima_estimado(ciudad, temporada):
    clima = full_df[(full_df["ciudad"] == ciudad) & (full_df["temporada"] == temporada)]
    if clima.empty:
        return None
    return clima[["temp_max", "temp_min", "precipitacion", "desc_actual", "humedad_actual"]].mean(numeric_only=True).round(1).to_dict()

def get_eventos(ciudad, temporada):
    eventos = full_df[(full_df["ciudad"] == ciudad) & (full_df["temporada"] == temporada)]
    eventos = eventos.dropna(subset=["evento_nombre", "evento_categoria", "evento_desc", "fecha"]).drop_duplicates()
    eventos = eventos[eventos["evento_nombre"] != "sin_evento"].head(3)
    if eventos.empty:
        return None
    return eventos.to_dict(orient="records")

def get_precio_vuelo(origen, destino):
    vuelos = full_df[(full_df["origin_city"] == origen) & (full_df["ciudad"] == destino)]
    if vuelos.empty:
        return None
    info = vuelos[["flight_price", "flight_duration_hr", "airline", "stops", "class"]].dropna().mean(numeric_only=True).round(1).to_dict()
    info["airline"] = vuelos["airline"].mode()[0]
    info["stops"] = vuelos["stops"].mode()[0]
    info["class"] = vuelos["class"].mode()[0]
    return info

def get_hotel(ciudad):
    hoteles = full_df[full_df["ciudad"] == ciudad]
    hoteles = hoteles.dropna(subset=["hotel_name", "estimated_price_eur_y", "hotel_type", "category", "hotel_type_1", "distance_to_city_center_km"])
    if hoteles.empty:
        return None
    hotel = hoteles.sort_values("estimated_price_eur_y").head(1)
    return hotel[["hotel_name", "estimated_price_eur_y", "hotel_type", "category", "hotel_type_1", "distance_to_city_center_km"]].to_dict(orient="records")[0]

# ========================
# 3. Input de usuario
# ========================
input_dict = {
    "perfil_viajero_n": "Soltero",
    "entornos_n": "Montaña",
    "clasificacion_destino_n": "Naturaleza",
    "temporada_n": "Invierno",
    "origin_city": "Madrid",
    "class_n": "Económica",
    "estimated_price_eur_x": 180,
    "estimated_price_eur_y": 95,
    "distance_to_city_center_km": 4.0
}

X_user = construir_input_usuario(input_dict, columnas_modelo)

# ========================
# 4. Predicción
# ========================
probs = model.predict_proba(X_user)[0]
top_indices = np.argsort(probs)[::-1]
top_labels = model.classes_[top_indices]
top_ciudades = [id_to_ciudad[str(lbl)] for lbl in top_labels]

# ========================
# 5. Mostrar resultados
# ========================
print("🏝️ Top 5 ciudades recomendadas:")
mostradas = 0
sin_eventos_mostrado = False

for ciudad in top_ciudades:
    if mostradas >= 5:
        break

    clima = get_clima_estimado(ciudad, input_dict["temporada_n"])
    eventos = get_eventos(ciudad, input_dict["temporada_n"])
    vuelo = get_precio_vuelo(input_dict["origin_city"], ciudad)
    hotel = get_hotel(ciudad)

    if None in (clima, vuelo, hotel):
        continue

    if eventos is None:
        if sin_eventos_mostrado:
            continue
        sin_eventos_mostrado = True

    mostradas += 1
    print(f"\n🌍 Ciudad: {ciudad}")
    print("☁️ Clima estimado:", clima)
    if eventos:
        print("🎫 Eventos:")
        for e in eventos:
            print(f" - {e['evento_nombre']} ({e['evento_categoria']}): {e['evento_desc']} - {e['fecha']}")
    else:
        print("🎫 Sin eventos disponibles.")
    print("✈️ Vuelo:", vuelo)
    print("🏨 Hotel:", hotel)

if mostradas == 0:
    print("\n⚠️ No se encontraron ciudades con información completa.")

🏝️ Top 5 ciudades recomendadas:

🌍 Ciudad: accra
☁️ Clima estimado: {'temp_max': 32.6, 'temp_min': 24.5, 'precipitacion': 0.9, 'humedad_actual': 67.0}
🎫 Sin eventos disponibles.
✈️ Vuelo: {'flight_price': 804.4, 'flight_duration_hr': 4.6, 'airline': 'Aeroméxico', 'stops': 'Directo', 'class': 'Económica'}
🏨 Hotel: {'hotel_name': 'Elite Accra Residence', 'estimated_price_eur_y': 209.13, 'hotel_type': 'Resort', 'category': 3.0, 'hotel_type_1': ' Ubicado en destinos turísticos, con actividades recreativas.', 'distance_to_city_center_km': 0.27}

🌍 Ciudad: honolulu
☁️ Clima estimado: {'temp_max': 18.4, 'temp_min': 10.1, 'precipitacion': 0.0, 'humedad_actual': 78.0}
🎫 Eventos:
 - HANDLING-SAT MAT (Desconocido): Sin descripción - 2025-12-13
 - HANDLING-SAT MAT (Desconocido): Sin descripción - 2025-12-13
 - HANDLING-SAT EVE (Desconocido): Sin descripción - 2025-12-13
✈️ Vuelo: {'flight_price': 1087.8, 'flight_duration_hr': 10.4, 'airline': 'Aeroméxico', 'stops': '1 escala', 'class': 'Económica'